In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.1.3
3.2.1


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator, 
    perform_experiments,
    Samplers, 
)
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.analysis import (feature_scoring, pairs_plotting)

from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x0000020045F561F0>)
ScalarOutcome('Dike Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs'), function=<function sum_over at 0x0000020045F561F0>)
ScalarOutcome('RfR Investment Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x0000020045F561F0>)
ScalarOutcome('Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x0000020045F561F0>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function

# Putting the outcomes in a dataframe

In [13]:
with open('saved_experiments.pkl', 'rb') as file:
    experiments = pickle.load(file)
    
with open('saved_outcomes.pkl', 'rb') as file:
    outcomes = pickle.load(file)

In [51]:
# Store all outcome names in a list
outcome_names = list(outcomes.keys())
print(outcome_names)


['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs']


In [ ]:
# Make a Dataframe with the Outcomes on the columns and the Uncertainties as rows
results = {}
for i in outcome_names:
    Si = sobol.analyze(problem, outcomes[i], calc_second_order=True, print_to_console=True)
    results[i] = Si

In [52]:
results = {}
for i in outcome_names:
    Si = sobol.analyze(problem, outcomes[i], calc_second_order=True, print_to_console=True)
    results[i] = Si


C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                               ST   ST_conf
A.0_ID flood wave shape  0.001555  0.000591
A.1_Bmax                 0.000000  0.000000
A.1_Brate                0.000000  0.000000
A.1_pfail                0.985283  0.069963
A.2_Bmax                 0.000000  0.000000
A.2_Brate                0.000000  0.000000
A.2_pfail                0.000000  0.000000
A.3_Bmax                 0.000000  0.000000
A.3_Brate                0.000000  0.000000
A.3_pfail                0.000000  0.000000
A.4_Bmax                 0.000000  0.000000
A.4_Brate                0.000000  0.000000
A.4_pfail                0.000000  0.000000
A.5_Bmax                 0.000000  0.000000
A.5_Brate                0.000000  0.000000
A.5_pfail                0.000000  0.000000
discount rate 0          0.012768  0.001521
discount rate 1          0.012664  0.001647
discount rate 2          0.012649  0.001736
                               S1   S1_conf
A.0_ID flood wave shape -0.002125  0.004049
A.1_Bmax                 0.00000

C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                               ST   ST_conf
A.0_ID flood wave shape  0.001815  0.000700
A.1_Bmax                 0.000000  0.000000
A.1_Brate                0.000000  0.000000
A.1_pfail                0.999837  0.059502
A.2_Bmax                 0.000000  0.000000
A.2_Brate                0.000000  0.000000
A.2_pfail                0.000000  0.000000
A.3_Bmax                 0.000000  0.000000
A.3_Brate                0.000000  0.000000
A.3_pfail                0.000000  0.000000
A.4_Bmax                 0.000000  0.000000
A.4_Brate                0.000000  0.000000
A.4_pfail                0.000000  0.000000
A.5_Bmax                 0.000000  0.000000
A.5_Brate                0.000000  0.000000
A.5_pfail                0.000000  0.000000
discount rate 0          0.000000  0.000000
discount rate 1          0.000000  0.000000
discount rate 2          0.000000  0.000000
                               S1   S1_conf
A.0_ID flood wave shape -0.002379  0.004006
A.1_Bmax                 0.00000

C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


...

                               ST   ST_conf
A.0_ID flood wave shape  0.015317  0.004903
A.1_Bmax                 0.031840  0.020182
A.1_Brate                0.000780  0.001065
A.1_pfail                0.220927  0.062751
A.2_Bmax                 0.007602  0.005851
A.2_Brate                0.000431  0.000694
A.2_pfail                0.127418  0.044498
A.3_Bmax                 0.013426  0.013199
A.3_Brate                0.000220  0.000208
A.3_pfail                0.189513  0.071575
A.4_Bmax                 0.000390  0.000410
A.4_Brate                0.000045  0.000051
A.4_pfail                0.073286  0.027288
A.5_Bmax                 0.000000  0.000000
A.5_Brate                0.000000  0.000000
A.5_pfail                0.834330  0.112486
discount rate 0          0.000000  0.000000
discount rate 1          0.000000  0.000000
discount rate 2          0.000000  0.000000
                               S1   S1_conf
A.0_ID flood wave shape -0.006770  0.011056
A.1_Bmax                 0.00715

C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\analyze\sobol.py:141: RuntimeWarning: invalid value encountered in divide
  Y = (Y - Y.mean()) / Y.std()


                         ST  ST_conf
A.0_ID flood wave shape NaN      NaN
A.1_Bmax                NaN      NaN
A.1_Brate               NaN      NaN
A.1_pfail               NaN      NaN
A.2_Bmax                NaN      NaN
A.2_Brate               NaN      NaN
A.2_pfail               NaN      NaN
A.3_Bmax                NaN      NaN
A.3_Brate               NaN      NaN
A.3_pfail               NaN      NaN
A.4_Bmax                NaN      NaN
A.4_Brate               NaN      NaN
A.4_pfail               NaN      NaN
A.5_Bmax                NaN      NaN
A.5_Brate               NaN      NaN
A.5_pfail               NaN      NaN
discount rate 0         NaN      NaN
discount rate 1         NaN      NaN
discount rate 2         NaN      NaN
                         S1  S1_conf
A.0_ID flood wave shape NaN      NaN
A.1_Bmax                NaN      NaN
A.1_Brate               NaN      NaN
A.1_pfail               NaN      NaN
A.2_Bmax                NaN      NaN
A.2_Brate               NaN      NaN
A

C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\analyze\sobol.py:141: RuntimeWarning: invalid value encountered in divide
  Y = (Y - Y.mean()) / Y.std()


                         ST  ST_conf
A.0_ID flood wave shape NaN      NaN
A.1_Bmax                NaN      NaN
A.1_Brate               NaN      NaN
A.1_pfail               NaN      NaN
A.2_Bmax                NaN      NaN
A.2_Brate               NaN      NaN
A.2_pfail               NaN      NaN
A.3_Bmax                NaN      NaN
A.3_Brate               NaN      NaN
A.3_pfail               NaN      NaN
A.4_Bmax                NaN      NaN
A.4_Brate               NaN      NaN
A.4_pfail               NaN      NaN
A.5_Bmax                NaN      NaN
A.5_Brate               NaN      NaN
A.5_pfail               NaN      NaN
discount rate 0         NaN      NaN
discount rate 1         NaN      NaN
discount rate 2         NaN      NaN
                         S1  S1_conf
A.0_ID flood wave shape NaN      NaN
A.1_Bmax                NaN      NaN
A.1_Brate               NaN      NaN
A.1_pfail               NaN      NaN
A.2_Bmax                NaN      NaN
A.2_Brate               NaN      NaN
A

In [55]:
sobol_results = {}
for policy in experiments.policy.unique():
    logical = experiments.policy == policy
    y = results['reliability'][logical]
    indices = analyze(problem, y)
    sobol_results[policy] = indices

KeyError: 'reliability'

KeyError: 'ST'

In [54]:
print(scores_filtered)

{'ST': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan]), 'ST_conf': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan]), 'S1': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan]), 'S1_conf': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])}


In [18]:
df = pd.DataFrame.from_dict(results)
display(df)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
S1,"[-0.002125130779956923, 0.0, 0.0, 0.9573080527...","[-0.0023787091775002295, 0.0, 0.0, 0.997897039...","[-0.0012195643225137805, 0.0036369296742197192...","[5.6187414403827506e-05, 0.0030012561359881854...","[0.002403448081323714, 0.003950170091335157, -...","[0.0034618222385297964, 0.0051564685348806745,...","[-0.00106856407958823, -0.0049565692339674575,...","[-0.0003532163332392836, -0.004335816685487215...","[-0.005543252147102582, 0.005448736535254986, ...","[-0.006770140785447941, 0.007149846768658483, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
S1_conf,"[0.003610688644735787, 0.0, 0.0, 0.06046390923...","[0.004496467044253288, 0.0, 0.0, 0.07637813813...","[0.0078948704757686, 0.011140941666385837, 0.0...","[0.0075643520501994166, 0.011736744548245063, ...","[0.0058327561725574585, 0.013573331079406052, ...","[0.006374728160568194, 0.013530690004111195, 0...","[0.011545966362571648, 0.011218212281853235, 0...","[0.014363175078888755, 0.010646220546232705, 0...","[0.00837560038299756, 0.010407685751030899, 0....","[0.009025691709235974, 0.01475204850085182, 0....","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
ST,"[0.0015550592738450204, 0.0, 0.0, 0.9852828198...","[0.001814719044908368, 0.0, 0.0, 0.99983689774...","[0.005098408491698487, 0.02334969434323253, 9....","[0.00492970105642332, 0.025907337052558998, 0....","[0.00526567124415722, 0.03095052724025968, 0.0...","[0.005901999803294875, 0.030001339432205943, 0...","[0.015245589375301065, 0.03743994676283876, 0....","[0.01787307358709154, 0.03551098703667767, 0.0...","[0.013801008024050923, 0.030587949707398063, 0...","[0.015317273983523987, 0.03183994293172672, 0....","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
ST_conf,"[0.0006096133239021526, 0.0, 0.0, 0.0659200745...","[0.0006324107150154205, 0.0, 0.0, 0.0692180869...","[0.0017582255808350719, 0.01248551790537973, 0...","[0.001487280722898329, 0.012803938956838986, 0...","[0.0015851637048396347, 0.01429524886680732, 0...","[0.0016661282452825016, 0.01097674680548697, 0...","[0.0033414546458263327, 0.019444826782744628, ...","[0.003965954511661885, 0.016725725682170588, 0...","[0.004721082738850072, 0.02444121021472599, 0....","[0.0050833320889727505, 0.02184152208030701, 0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
S2,"[[nan, 0.0005125664747997983, 0.00051256647479...","[[nan, 0.0007909348557796664, 0.00079093485577...","[[nan, 0.0004855031719394544, -0.0005471247274...","[[nan, -0.00015618923776800683, -0.00184919966...","[[nan, 0.0014968181604600233, 0.00088961539298...","[[nan, 0.0006521466555948164, -1.0152296273536...","[[nan, 0.007131881466374562, 0.005172020782608...","[[nan, 0.00605604076348694, 0.0041355702202927...","[[nan, 0.016131767094972778, 0.013247815427265...","[[nan, 0.019840397226009453, 0.016334911237016...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
S2_conf,"[[nan, 0.004529436437517812, 0.004529436437517...","[[nan, 0.005631202383157964, 0.005631202383157...","[[nan, 0.013175593455443384, 0.013560869288637...","[[nan, 0.011510450391506074, 0.011846553167674...","[[nan, 0.010360891210700019, 0.010325375649058...","[[nan, 0.011372057173999903, 0.011367826641867...","[[nan, 0.024718312840773016, 0.024317988161669...","[[nan, 0.02943827609153807, 0.0297950560985766...","[[nan, 0.017475293469675742, 0.016895968418411...","[[nan, 0.019601531495147102, 0.018570786520333...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[

In [22]:
for i in df["A.1 Total Costs"]:
    print(i[0])

-0.002125130779956923
0.003610688644735787
0.0015550592738450204
0.0006096133239021526
[       nan 0.00051257 0.00051257 0.0034736  0.00051257 0.00051257
 0.00051257 0.00051257 0.00051257 0.00051257 0.00051257 0.00051257
 0.00051257 0.00051257 0.00051257 0.00051257 0.00079428 0.00070661
 0.00055488]
[       nan 0.00452944 0.00452944 0.00519799 0.00452944 0.00452944
 0.00452944 0.00452944 0.00452944 0.00452944 0.00452944 0.00452944
 0.00452944 0.00452944 0.00452944 0.00452944 0.0046767  0.00467939
 0.00470727]
